In [25]:
import sqlite3
import openpyxl
from mapping import map
import os
import datetime
import pandas as pd

with open('create_table.sql', 'r') as file:
    create_table_query = file.read()
with open('insert.sql', 'r') as file:
    insert_query = file.read()

workbook_path = 'New cases 0223.xlsx'
workbook = openpyxl.load_workbook(workbook_path, data_only=True)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [26]:
db_dir= 'patient_cases.db'
if os.path.exists(db_dir):
    now = datetime.datetime.now()
    db_dir_old = 'old db as of '+ str(now)
    os.rename(db_dir, db_dir_old)
else:
    message = f"The file '{db_dir}' does not exist."

conn = sqlite3.connect('patient_cases.db')
cursor = conn.cursor()
cursor.execute(create_table_query)

In [27]:
def extract_data_from_sheet(sheet):
    return [sheet.cell(row=pos[0], column=pos[1]).value for field, pos in map.items()]

In [29]:
for sheet_name in workbook.sheetnames:
    if sheet_name[0] in ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9'):
        sheet = workbook[sheet_name]
        case_data = extract_data_from_sheet(sheet)
        if len(case_data) != insert_query.count('?'):
            print(f"Mismatch between number of placeholders and data in sheet {sheet_name}. Skipping.")
            continue

        try:
            cursor.execute(insert_query, case_data)
        except Exception as e:
            print(f"Error inserting data for sheet {sheet_name}: {e}")
            continue
conn.commit()


In [22]:
query = "SELECT * FROM patient_cases WHERE first_bed_type IS NOT NULL AND first_bed_type <> 0;"
df = pd.read_sql_query(query, conn)
conn.close()
df

,case_id,arrival_severity,gender,age,injury_icd10_1,injury_icd10_2,injury_icd10_3,injury_icd10_4,injury_ais_1,injury_ais_2,...,tenth_bed_start,tenth_bed_end,tenth_bed_PRIMARY,tenth_bed_SECONDARY,tenth_bed_TERTIARY,tenth_bed_POST_ACUTE_4,tenth_bed_POST_ACUTE_5,tenth_bed_OTHER_SPECIAL_1,tenth_bed_OTHER_SPECIAL_2,tenth_bed_OTHER_SPECIAL_3
0,1,moderate,F,23,S82. 109A,S51.819A,None,None,4,None,...,None,None,None,None,None,None,None,None,None,None
1,2,serious,M,20,S58.12,None,None,None,3,None,...,None,None,None,None,None,None,None,None,None,None
2,3,Minor,M,21,S06,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,4,Moderate,M,34,T50.3,None,None,None,3,None,...,None,None,None,None,None,None,None,None,None,None
4,5,Serious,Medical Status,23,S82.219B,S82.409B,None,None,3,3,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,169,Severe,M,29,S36.438A,S36.63XA,S35.514A,None,4,4,...,None,None,None,None,None,None,None,None,None,None
142,170,Serious,M,21,S02.652B,S02.102A,S42.032B,None,3,3,...,None,None,None,None,None,None,None,None,None,None
143,171,Severe,M,28,S27.2XXA,S22.42XA,S27.331A,S27.321A,4,3,...,None,None,None,None,None,None,None,None,None,None
144,172,Serious,M,19,S02.2XXB,S02.40DA,S01.312A,None,3,2,...,None,None,None,None,None,None,None,None,None,None


In [30]:
cleaned = df[[
    "case_id", "arrival_severity",
    "first_bed_type", "first_bed_hours",
    "second_bed_type", "second_bed_hours", 
    "third_bed_type", "third_bed_hours", 
    "fourth_bed_type", "fourth_bed_hours", 
    "fifth_bed_type", "fifth_bed_hours", 
    "sixth_bed_type", "sixth_bed_hours", 
    "seventh_bed_type", "seventh_bed_hours", 
    "eighth_bed_type", "eighth_bed_hours", 
    "ninth_bed_type", "ninth_bed_hours", 
    "tenth_bed_type", "tenth_bed_hours"
]]

In [31]:
cleaned

,case_id,arrival_severity,first_bed_type,first_bed_hours,second_bed_type,second_bed_hours,third_bed_type,third_bed_hours,fourth_bed_type,fourth_bed_hours,...,sixth_bed_type,sixth_bed_hours,seventh_bed_type,seventh_bed_hours,eighth_bed_type,eighth_bed_hours,ninth_bed_type,ninth_bed_hours,tenth_bed_type,tenth_bed_hours
0,1,moderate,Med/Surg,48,Medical Hotel/Equivalent,1008.0,None,NaN,None,NaN,...,None,None,None,None,None,None,None,None,None,None
1,2,serious,ICU/Critical Care,24,Med/Surg,48.0,Medical Hotel/Equivalent,720.0,None,NaN,...,None,None,None,None,None,None,None,None,None,None
2,3,Minor,Med/Surg,24,Medical Hotel/Equivalent,720.0,None,NaN,None,NaN,...,None,None,None,None,None,None,None,None,None,None
3,4,Moderate,Med/Surg,24,Medical Hotel/Equivalent,168.0,None,NaN,None,NaN,...,None,None,None,None,None,None,None,None,None,None
4,5,Serious,Med/Surg,240,Acute Rehabilitation Unit (ARU),336.0,Medical Hotel/Equivalent,672.0,None,NaN,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,169,Severe,ICU/Critical Care,96,Med/Surg,72.0,None,NaN,None,NaN,...,None,None,None,None,None,None,None,None,None,None
142,170,Serious,ICU/Critical Care,216,Med/Surg,240.0,None,NaN,None,NaN,...,None,None,None,None,None,None,None,None,None,None
143,171,Severe,Med/Surg,144,Medical Hotel/Equivalent,168.0,None,NaN,None,NaN,...,None,None,None,None,None,None,None,None,None,None
144,172,Serious,Med/Surg,24,Medical Hotel/Equivalent,168.0,None,NaN,None,NaN,...,None,None,None,None,None,None,None,None,None,None


In [32]:
cleaned.to_csv("cleaned_patient_data.csv", index = False)
df.to_csv("patient_data.csv", index = False)
